In [402]:
import numpy as np
from numpy import tril, eye, diag
from numpy.linalg import eig, norm, inv, det, solve

import scipy
# from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

import time

import matplotlib.pyplot as plt
%matplotlib inline

from prettytable import PrettyTable

# debug constants
DEBUG = False


In [403]:
def find_iteration_matrix_guass_seidel(A):
    """
    Find iteration matrix of an input square matrix A for Gauss Seidel

    M = I - (L + D)^{-1} A

    :param A:
    :return:
    """
    n, _ = A.shape

    # FIXME: use sparse versions...
    # A_LD = tril(A)
    # A_LD_inv = inv(A_LD)
    A_LD = scipy.sparse.tril(A)
    A_LD_inv = scipy.sparse.linalg.inv(A_LD)

    if DEBUG:
        print(f"L + D:\n{A_LD}")
        print(f"(L + D)^-1:\n {A_LD_inv}")

    M = eye(n) - A_LD_inv @ A

    if DEBUG:
        print(f"M:\n{M}")

    return M


In [404]:
def find_spectral_radius(M):
    """
    Given an iteration matrix M, finds the spectral radius

    rho(M) = max{|lambda| where lambda is an eigenvalue of M}

    :param M:
    :return:
    """

    M_eigenvalues, _ = eig(M)

    if DEBUG:
        print(f"M eigenvalues:\n {M_eigenvalues}")

    return max(np.abs(M_eigenvalues))


In [405]:
# problem 2
# part a

# A is a square matrix
A = np.array([[1, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

A_inv = inv(A)
A_det = det(A)

print(f"A inverse:\n {A_inv}")
print(f"A det: {A_det}")


A inverse:
 [[3. 2. 1.]
 [2. 2. 1.]
 [1. 1. 1.]]
A det: 1.0


In [406]:
# part b
M = find_iteration_matrix_guass_seidel(A)
rho_M = find_spectral_radius(M)

print(f"rho(M) = {rho_M}")


rho(M) = 0.75


In [407]:
# part C
M_inf_norm = norm(M, ord=inf)
M_1_norm = norm(M, ord=1)
M_2_norm = norm(M, ord=2)

print(f"||M||_inf = {M_inf_norm}")
print(f"||M||_1 = {M_1_norm}")
print(f"||M||_2 = {M_2_norm}")


||M||_inf = 1.0
||M||_1 = 1.75
||M||_2 = 1.1841130945790346


In [408]:
# problem 3
# part a

f = lambda x: 4 * np.pi**2 * np.sin(2 * np.pi * x)

def form_sparse_lin_sys(N):
    twos_diag = 2*np.ones((N, ))
    ones_off_diag = np.ones((N - 1, ))
    negative_ones_off_diag = -ones_off_diag

    # FIXME: old implementation
    # twos_main_diag = scipy.sparse.diags(twos_diag)
    # ones_sub_diag = scipy.sparse.diags(ones_off_diag, -1)
    # ones_super_diag = scipy.sparse.diags(ones_off_diag, 1)
    # A = twos_main_diag - ones_sub_diag  - ones_super_diag

    # specify all diagonals at once
    # need csr or csc matrix
    A = scipy.sparse.diags([negative_ones_off_diag, twos_diag, negative_ones_off_diag], [-1, 0, 1], format="csr")

    h = 1 / (N + 1)

    b = np.zeros((N, ))
    for i in range(N):
        b[i] = h ** 2 * f((i + 1) * h)

    return A, b


In [409]:
N = 3
A, b = form_sparse_lin_sys(3)

print(f"N: {N}")
print(f"A:\n{A}")
print(f"A:\n{A.toarray()}")
print(f"b:\n{b}")


N: 3
A:
  (0, 0)	2.0
  (0, 1)	-1.0
  (1, 0)	-1.0
  (1, 1)	2.0
  (1, 2)	-1.0
  (2, 1)	-1.0
  (2, 2)	2.0
A:
[[ 2. -1.  0.]
 [-1.  2. -1.]
 [ 0. -1.  2.]]
b:
[ 2.46740110e+00  3.02169486e-16 -2.46740110e+00]


In [410]:
# part b
def solve_sparse_system(A, b):
    return spsolve(A, b)


In [411]:
# part c
def jacobi(A, b, num_iter=50):

    # A is a square matrix
    n, _ = A.shape

    # zero vector as initial guess
    x_curr = np.zeros((n, ))

    # returns a vector with just the diagonal entries
    # D = diag(A)
    D = A.diagonal()

    # keep track of the 2-norm of the error (||x - x_k||_2)
    x_true = solve_sparse_system(A, b)
    errors = np.zeros((num_iter + 1, ))
    errors[0] = norm(x_true - x_curr)

    for i in range(num_iter):
        curr_residual = b - A @ x_curr

        # inverse of D is just has the diagonal entries inverted (also diagonal matrix)
        x_curr = x_curr + curr_residual / D

        # calculate errors at each iteration
        errors[i + 1] = norm(x_true - x_curr)

    return x_curr, errors


def gauss_seidel(A, b, num_iter=50):
    # A is a square matrix
    n, _ = A.shape

    # zero vector as initial guess
    x_curr = np.zeros((n, ))

    # keep track of the 2-norm of the error (||x - x_k||_2)
    x_true = solve_sparse_system(A, b)
    errors = np.zeros((num_iter + 1, ))
    errors[0] = norm(x_true - x_curr)

    for k in range(num_iter):
        for i in range(n):
            # FIXME: need to go up to i, but NOT including i == j
            # when the slice is empty ([0:0]), then any operations will return 0s...
            x_curr[i] = (b[i] - A[i, 0:i] @ x_curr[0:i]
                    - A[i, (i + 1):n] @ x_curr[(i + 1):n]) \
                   / A[i, i]

        # calculate errors at each iteration
        errors[k + 1] = norm(x_true - x_curr)

    return x_curr, errors


In [412]:
# part d
# iteration matrix for Gauss-Seidel is given above
# spectral radius is also given above
def find_iteration_matrix_jacobi(A):
    """
    Find iteration matrix of an input square matrix A for Jacobi

    M = I - D^{-1} A

    :param A:
    :return:
    """
    n, _ = A.shape

    # D = diag(A)
    D = A.diagonal()

    # inverse of D has just the reciprocal of the diagonal entries
    D_inv = 1 / D

    if DEBUG:
        print(f"D:\n{D}")
        print(f"D^-1:\n {D_inv}")

    D_inv_A = np.zeros(A.shape)
    D_inv_A = csr_matrix(A.shape)
    for i in range(n):
        D_inv_A[i, :] = D_inv[i] * A[i, :]

    M = eye(n) - D_inv_A

    if DEBUG:
        print(f"M:\n{M}")

    return M


In [413]:
# part e
N_arr = np.array([10, 20, 40, 80, 160])
N_arr_size = len(N_arr)
num_iter = 200

errors_jacobi_array = np.zeros((N_arr_size, num_iter + 1))
errors_gauss_seidel_array = np.zeros((N_arr_size, num_iter + 1))

rho_jacobi_array = np.zeros((N_arr_size, ))
rho_gauss_seidel_array = np.zeros((N_arr_size, ))

relative_error_initial_matrix_jacobi = np.zeros((N_arr_size, num_iter + 1))
relative_error_initial_matrix_gauss_seidel = np.zeros((N_arr_size, num_iter + 1))

for i, N in enumerate(N_arr):
    # part a
    A, b = form_sparse_lin_sys(N)

    # part b
    x_true = spsolve(A, b)

    # part c
    x_jacobi, errors_jacobi = jacobi(A, b, num_iter=num_iter)
    x_gauss_seidel, errors_gauss_seidel = gauss_seidel(A, b, num_iter=num_iter)

    errors_jacobi_array[i, :] = errors_jacobi[:]
    errors_gauss_seidel_array[i, :] = errors_gauss_seidel[:]

    # part d
    M_jacobi = find_iteration_matrix_jacobi(A)
    rho_jacobi = find_spectral_radius(M_jacobi)

    M_gauss_seidel = find_iteration_matrix_guass_seidel(A)
    rho_gauss_seidel = find_spectral_radius(M_gauss_seidel)

    rho_jacobi_array[i] = rho_jacobi
    rho_gauss_seidel_array[i] = rho_gauss_seidel

    # part e
    # compute relative error to initial guess
    relative_error_initial_vector_jacobi = np.zeros((num_iter + 1, ))
    relative_error_initial_vector_jacobi[0] = None
    relative_error_initial_vector_jacobi[1:] = errors_jacobi[1:] / errors_jacobi[0:-1]
    relative_error_initial_matrix_jacobi[i, :] = relative_error_initial_vector_jacobi

    relative_error_initial_vector_gauss_seidel = np.zeros((num_iter + 1, ))
    relative_error_initial_vector_gauss_seidel[0] = None
    relative_error_initial_vector_gauss_seidel[1:] = errors_gauss_seidel[1:] / errors_gauss_seidel[0:-1]
    relative_error_initial_matrix_gauss_seidel[i, :] = relative_error_initial_vector_gauss_seidel

# print table out
table = PrettyTable()
table.field_names = ["Iteration number (i)",
                     "Jacobi Error ||x* - x_k||_2",
                     "Jacobi Eta_k ||e_k|| / ||e_0||",
                     "Gauss-Seidel Error ||x* - x_k||_2",
                     "Gauss-Seidel Eta_k ||e_k|| / ||e_0||"]

N_array_index = 0

errors_jacobi = errors_jacobi_array[N_array_index, :]
relative_error_initial_vector_jacobi = relative_error_initial_matrix_jacobi[N_array_index, :]

errors_gauss_seidel = errors_gauss_seidel_array[N_array_index, :]
relative_error_initial_vector_gauss_seidel = relative_error_initial_matrix_gauss_seidel[N_array_index, :]

table.add_rows(
    [
        [
            i,
             f"{errors_jacobi[i]: e}",
             f"{relative_error_initial_vector_jacobi[i]: e}",
             f"{errors_gauss_seidel[i]: e}",
             f"{relative_error_initial_vector_gauss_seidel[i]: e}"
         ]
        for i in range(num_iter + 1)
    ]
)

print(table)



+----------------------+-----------------------------+--------------------------------+-----------------------------------+--------------------------------------+
| Iteration number (i) | Jacobi Error ||x* - x_k||_2 | Jacobi Eta_k ||e_k|| / ||e_0|| | Gauss-Seidel Error ||x* - x_k||_2 | Gauss-Seidel Eta_k ||e_k|| / ||e_0|| |
+----------------------+-----------------------------+--------------------------------+-----------------------------------+--------------------------------------+
|          0           |         2.410026e+00        |               nan              |            2.410026e+00           |                  nan                 |
|          1           |         2.027443e+00        |          8.412535e-01          |            1.849705e+00           |             7.675042e-01             |
|          2           |         1.705593e+00        |          8.412535e-01          |            1.408436e+00           |             7.614381e-01             |
|          3          

In [414]:
# part f
for i in range(N_arr_size):
    print(f"Jacobi")
    print(f"array size {N_arr[i]}, rho(M) = {rho_jacobi_array[i]}")

    print(f"Gauss-Seidel")
    print(f"array size {N_arr[i]}, rho(M) = {rho_gauss_seidel_array[i]}")



Jacobi
array size 10, rho(M) = 0.959492973614498
Gauss-Seidel
array size 10, rho(M) = 0.9206267664155902
Jacobi
array size 20, rho(M) = 0.9888308262251292
Gauss-Seidel
array size 20, rho(M) = 0.977786402893069
Jacobi
array size 40, rho(M) = 0.9970658011837408
Gauss-Seidel
array size 40, rho(M) = 0.9941402118901728
Jacobi
array size 80, rho(M) = 0.9992479525042306
Gauss-Seidel
array size 80, rho(M) = 0.9984964705838957
Jacobi
array size 160, rho(M) = 0.9998096274980756
Gauss-Seidel
array size 160, rho(M) = 0.9996192912378364


In [415]:
# scratch code

# N = 3
# twos_diag = 2*np.ones((N,))
# ones_off_diag = np.ones((N-1,))
# A = np.diag(twos_diag) - np.diag(ones_off_diag,1)  - np.diag(ones_off_diag,-1)
# A = np.diag(twos_diag) - np.diag(ones_off_diag,1)  - np.diag(ones_off_diag,-1)
# print(f"A: {A}")

a = np.array([1, 2])
print(a[0:1])

b = np.array([1, 2])

print(a[0:0] @ b[0:0])


[1]
0
